In [1]:
!pip install diffusers transformers torch opencv-python numpy Pillow

In [2]:
import cv2
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFont
from diffusers import StableDiffusionPipeline
import torch

# Function to generate images using Stable Diffusion
def generate_images_from_prompt(prompt, num_images=2, width=1280, height=720):
    """
    Generate images from a text prompt using Stable Diffusion (Hugging Face).
    Falls back to placeholder images if generation fails.
    Returns a list of file paths to generated images.
    """
    output_paths = []
    try:
        # Initialize Stable Diffusion model
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            use_auth_token="your_huggingface_token"  # Replace with your token
        )
        pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", "CUDA" if torch.cuda.is_available() else "CPU")

        for i in range(num_images):
            # Generate image
            image = pipe(prompt, width=width, height=height).images[0]
            output_path = f"generated_image_{i+1}.png"
            image.save(output_path)
            output_paths.append(output_path)
    except Exception as e:
        print(f"Error generating images with Stable Diffusion: {e}")
        print("Falling back to placeholder images...")
        # Fallback to placeholder images
        try:
            font = ImageFont.truetype("arial.ttf", 50)
        except:
            font = ImageFont.load_default()

        for i in range(num_images):
            colors = [(255, 100, 100), (100, 255, 100)]  # Reddish, Greenish
            img = Image.new("RGB", (width, height), color=colors[i % len(colors)])
            draw = ImageDraw.Draw(img)
            text = f"{prompt} (Image {i+1})"
            text_bbox = draw.textbbox((0, 0), text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            text_x = (width - text_width) // 2
            text_y = (height - text_height) // 2
            draw.text((text_x, text_y), text, fill=(255, 255, 255), font=font)
            output_path = f"generated_image_{i+1}.png"
            img.save(output_path)
            output_paths.append(output_path)
    return output_paths

# Parameters
prompt = "A majestic tiger in a jungle"  # Text prompt
output_file = "tiger_video.mp4"  # Output video file name
duration = 3  # Total video duration in seconds (2-5 seconds)
fps = 24  # Frames per second
width, height = 1280, 720  # Video resolution
transition_duration = 0.5  # Duration of crossfade in seconds
image_folder = r"C:\Users\SreeKeerthiReddyThat\Desktop\Image-generation\lvm\image_folder"  # Save generated images here

# Ensure image folder exists
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

# Debugging: Print current directory
print("Current working directory:", os.getcwd())
print("Saving generated images to:", os.path.abspath(image_folder))

# Generate images from prompt
image_files = generate_images_from_prompt(prompt, num_images=2, width=width, height=height)
image_files = [os.path.join(image_folder, os.path.basename(f)) for f in image_files]  # Update paths to image_folder
print("Generated image files:", image_files)

# Check if enough images were generated
if len(image_files) < 2:
    print("Error: Could not generate at least 2 images. Check prompt or generation function.")
    exit()

# Load and resize images
images = []
for img_path in image_files:
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Error: Failed to load image {img_path}. It may be corrupted or inaccessible.")
        continue
    images.append(cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA))
print(f"Successfully loaded {len(images)} images")

# Check if enough valid images were loaded
if len(images) < 2:
    print("Error: Could not load at least 2 valid images.")
    exit()

# Calculate frames per image
total_frames = int(duration * fps)
frames_per_image = total_frames // len(images)
transition_frames = int(transition_duration * fps)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
if not video.isOpened():
    print("Error: Could not initialize video writer. Try installing FFmpeg and using 'X264' codec.")
    exit()

# Generate video frames
for i in range(total_frames):
    current_image_idx = min(i // frames_per_image, len(images) - 1)
    next_image_idx = min(current_image_idx + 1, len(images) - 1)

    # Calculate transition progress
    frame_in_image = i % frames_per_image
    if frame_in_image < transition_frames and current_image_idx < len(images) - 1:
        # Crossfade transition
        alpha = frame_in_image / transition_frames
        beta = 1 - alpha
        frame = cv2.addWeighted(images[current_image_idx], beta, images[next_image_idx], alpha, 0.0)
    else:
        # Display current image
        frame = images[current_image_idx]

    video.write(frame)

# Release video writer
video.release()

# Clean up generated images (optional)
for img_path in image_files:
    try:
        os.remove(img_path)
    except:
        pass

print(f"Video saved as {output_file}")


c:\Users\SreeKeerthiReddyThat\Desktop\Image-generation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (c:\Users\SreeKeerthiReddyThat\Desktop\Image-generation\venv\Lib\site-packages\huggingface_hub\__init__.py)